In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobaccesskey"))

In [0]:
group_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/group.csv", header=True, inferSchema=True)

In [0]:
#group_data.show(20,False)

In [0]:
group_data.select("*").filter(col("grp_id").isin("GRP102", "GRP112", "GRP132")).show(5,False)

+-------+---------------+--------+------+--------------------------------------+--------+------+
|country|premium_written|zip_code|grp_id|grp_name                              |grp_type|city  |
+-------+---------------+--------+------+--------------------------------------+--------+------+
|India  |45000          |482049  |GRP102|HDFC Standard Life Insurance Co. Ltd. |Private |Mumbai|
|India  |57000          |482022  |GRP112|Reliance Nippon Life Insurance Company|Private |Mumbai|
|India  |57000          |482048  |GRP132|Cigna TTK                             |Private |Mumbai|
+-------+---------------+--------+------+--------------------------------------+--------+------+



In [0]:
#group_data.groupby(group_data.columns).count().where("count > 1").show() #check the duplicates

+-------+---------------+--------+------+--------+--------+----+-----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|count|
+-------+---------------+--------+------+--------+--------+----+-----+
+-------+---------------+--------+------+--------+--------+----+-----+



In [0]:
group_data = group_data.dropDuplicates()

In [0]:
#group_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in group_data.columns]).show()

+-------+---------------+--------+------+--------+--------+----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|
+-------+---------------+--------+------+--------+--------+----+
|      0|              0|       0|     0|       0|       0|   0|
+-------+---------------+--------+------+--------+--------+----+



In [0]:
#loading df into stagging
#stagging_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
#group_data.write.format("csv").option("header", "true").mode("overwrite").option("path",stagging_path).save()

In [0]:

output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
group_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/groupdatastagging.csv" % output_container_path)


True

In [0]:
"""
output_container_path = "abfss://stagingdatasets@careblobstore.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/wrangled_data_folder" 
data.write.format("json").mode("overwrite").save(output_blob_folder)

files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/staging_claims.json" % output_container_path)
"""